In [1]:
import rdkit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import mbuild as mb
import parmed
import torch
from torch import nn
import torch_geometric as tg
from torch_geometric.loader import DataLoader
import torch.utils.data
import shutil
import sys
import rdkit 
sys.path.insert(1, '~/egnn_cof/models/egnn_clean')
sys.path.insert(1, '~/egnn_cof')
# from easydict import EasyDict as edict
import torch
from torch import nn, optim
# My imports
from loader import *
from megnn import *
from utils import *

shutil.rmtree('./processed')

In [2]:
dat = Cloud_Point_Dataset(root='.')
for d in dat:
    print(d)
    break

Processing...
[16:39:40] UFFTYPER: Unrecognized atom type: *_ (3)
[16:39:40] UFFTYPER: Unrecognized atom type: *_ (5)
[16:39:41] Molecule does not have explicit Hs. Consider calling AddHs()
[16:39:41] UFFTYPER: Unrecognized atom type: *_ (0)
[16:39:41] UFFTYPER: Unrecognized atom type: *_ (3)
[16:39:41] Molecule does not have explicit Hs. Consider calling AddHs()
[16:39:41] UFFTYPER: Unrecognized atom type: *_ (0)
[16:39:41] UFFTYPER: Unrecognized atom type: *_ (3)
[16:39:41] Molecule does not have explicit Hs. Consider calling AddHs()
[16:39:41] UFFTYPER: Unrecognized atom type: *_ (0)
[16:39:41] UFFTYPER: Unrecognized atom type: *_ (6)
[16:39:41] Molecule does not have explicit Hs. Consider calling AddHs()
[16:39:41] UFFTYPER: Unrecognized atom type: *_ (0)
[16:39:41] UFFTYPER: Unrecognized atom type: *_ (3)
[16:39:41] Molecule does not have explicit Hs. Consider calling AddHs()
[16:39:41] UFFTYPER: Unrecognized atom type: *_ (0)
[16:39:41] UFFTYPER: Unrecognized atom type: *_ (3)
[1

PairData(edge_index_s=[2, 16], x_s=[16, 7], positions_s=[16, 3], n_nodes_s=[1], edge_index_t=[2, 4], x_t=[5, 7], positions_t=[5, 3], n_nodes_t=[1], enviro=[4], y=[1])


Done!


In [4]:
d.edge_index_s

tensor([[ 1,  2,  3,  4,  5,  6, 14,  7,  8, 12, 15,  9, 10, 11, 12, 13],
        [ 1,  2,  3,  4,  5,  6, 14,  7,  8, 12, 15,  9, 10, 11, 12, 13]])

In [26]:
# clear the processed dataset
try:
    shutil.rmtree('./processed')
except:
    pass

n_epochs  = 5
device = torch.device("cpu")
dtype = torch.float32
dat = Cloud_Point_Dataset(root='.')

batch_size = 32

dat.shuffle()
train_dataset = dat[:4000]
test_dataset = dat[4000:5000]
valid_dataset = dat[5000:]
train_loader = DataLoader(train_dataset, batch_size=batch_size, follow_batch=['x_s', 'x_t', 'positions_s', 'positions_t'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, follow_batch=['x_s', 'x_t','positions_s', 'positions_t'], shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, follow_batch=['x_s', 'x_t','positions_s', 'positions_t'], shuffle=False)

model = MEGNN(n_graphs=2, in_node_nf=len(dat.elements), in_edge_nf=0, hidden_nf=128, device=device, n_layers=7, coords_weight=1.0,
             attention=True, node_attr=1)
# model = PairEGNN(in_node_nf=len(dat.elements), in_edge_nf=0, hidden_nf=128, device=device, n_layers=7, coords_weight=1.0,
#              attention=True, node_attr=1)

optimizer = optim.Adam(model.parameters())
loss_func = nn.MSELoss()


def train():
    model.train()
    epoch_loss = 0
    for data in train_loader:  # Iterate in batches over the training dataset.
        dense_positions_s, atom_mask_s = tg.utils.to_dense_batch(data.positions_s, data.positions_s_batch)
        dense_positions_t, atom_mask_t = tg.utils.to_dense_batch(data.positions_t, data.positions_t_batch)
        batch_size_s, n_nodes_s, _ = dense_positions_s.size()
        batch_size_t, n_nodes_t, _ = dense_positions_t.size()
        atom_positions_s = dense_positions_s.view(batch_size_s * n_nodes_s, -1).to(device, dtype)
        atom_positions_t = dense_positions_t.view(batch_size_t * n_nodes_t, -1).to(device, dtype)
        edge_mask_s = atom_mask_s.unsqueeze(1) * atom_mask_s.unsqueeze(2)
        diag_mask = ~torch.eye(edge_mask_s.size(1), dtype=torch.bool).unsqueeze(0)
        edge_mask_s *= diag_mask
        edge_mask_s = edge_mask_s.view(batch_size_s * n_nodes_s * n_nodes_s, 1).to(device)
        edge_mask_t = atom_mask_t.unsqueeze(1) * atom_mask_t.unsqueeze(2)
        diag_mask = ~torch.eye(edge_mask_t.size(1), dtype=torch.bool).unsqueeze(0)
        edge_mask_t *= diag_mask
        edge_mask_t = edge_mask_t.view(batch_size_t * n_nodes_t * n_nodes_t, 1).to(device)
        atom_mask_s = atom_mask_s.view(batch_size_s * n_nodes_s, -1).to(device)
        atom_mask_t = atom_mask_t.view(batch_size_t * n_nodes_t, -1).to(device)
        one_hot_s, one_hot_s_mask = tg.utils.to_dense_batch(data.x_s, data.x_s_batch)
        one_hot_t, one_hot_s_mask = tg.utils.to_dense_batch(data.x_t, data.x_t_batch)
        one_hot_s = one_hot_s.view(batch_size_s * n_nodes_s, -1).to(device)
        one_hot_t = one_hot_t.view(batch_size_t * n_nodes_t, -1).to(device)
        edges_s = get_adj_matrix(n_nodes_s, batch_size_s, device)
        edges_t = get_adj_matrix(n_nodes_t, batch_size_t, device)
        label = data.y.to(device, dtype)
        
        # pred = model(h0_s=one_hot_s, h0_t=one_hot_t, edges_s=edges_s, edges_t=edges_t, edge_attr=None, node_mask_s=atom_mask_s, 
        #             edge_mask_s=edge_mask_s, n_nodes_s=n_nodes_s, node_mask_t=atom_mask_t, edge_mask_t=edge_mask_t, 
        #             n_nodes_t=n_nodes_t, x_s=atom_positions_s, x_t=atom_positions_t)
        pred = model(h0=[one_hot_s, one_hot_t], x=[atom_positions_s, atom_positions_t], all_edges=[edges_s, edges_t],
                         all_edge_attr=[None, None], node_masks=[atom_mask_s, atom_mask_t],
                         edge_masks=[edge_mask_s, edge_mask_t], n_nodes=[n_nodes_s, n_nodes_t])
        loss = loss_func(pred, label)
        epoch_loss += loss
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    return epoch_loss

def test(loader):
    model.eval()

    squared_error = []
    error = 0 
    for data in loader:  # Iterate in batches over the training/test dataset.
        dense_positions_s, atom_mask_s = tg.utils.to_dense_batch(data.positions_s, data.positions_s_batch)
        dense_positions_t, atom_mask_t = tg.utils.to_dense_batch(data.positions_t, data.positions_t_batch)
        batch_size_s, n_nodes_s, _ = dense_positions_s.size()
        batch_size_t, n_nodes_t, _ = dense_positions_t.size()
        atom_positions_s = dense_positions_s.view(batch_size_s * n_nodes_s, -1).to(device, dtype)
        atom_positions_t = dense_positions_t.view(batch_size_t * n_nodes_t, -1).to(device, dtype)
        edge_mask_s = atom_mask_s.unsqueeze(1) * atom_mask_s.unsqueeze(2)
        diag_mask = ~torch.eye(edge_mask_s.size(1), dtype=torch.bool).unsqueeze(0)
        edge_mask_s *= diag_mask
        edge_mask_s = edge_mask_s.view(batch_size_s * n_nodes_s * n_nodes_s, 1).to(device)
        edge_mask_t = atom_mask_t.unsqueeze(1) * atom_mask_t.unsqueeze(2)
        diag_mask = ~torch.eye(edge_mask_t.size(1), dtype=torch.bool).unsqueeze(0)
        edge_mask_t *= diag_mask
        edge_mask_t = edge_mask_t.view(batch_size_t * n_nodes_t * n_nodes_t, 1).to(device)
        atom_mask_s = atom_mask_s.view(batch_size_s * n_nodes_s, -1).to(device)
        atom_mask_t = atom_mask_t.view(batch_size_t * n_nodes_t, -1).to(device)
        one_hot_s, one_hot_s_mask = tg.utils.to_dense_batch(data.x_s, data.x_s_batch)
        one_hot_t, one_hot_s_mask = tg.utils.to_dense_batch(data.x_t, data.x_t_batch)
        one_hot_s = one_hot_s.view(batch_size_s * n_nodes_s, -1).to(device)
        one_hot_t = one_hot_t.view(batch_size_t * n_nodes_t, -1).to(device)
        edges_s = get_adj_matrix(n_nodes_s, batch_size_s, device)
        edges_t = get_adj_matrix(n_nodes_t, batch_size_t, device)
        label = data.y.to(device, dtype)
        
        # pred = model(h0_s=one_hot_s, h0_t=one_hot_t, edges_s=edges_s, edges_t=edges_t, edge_attr=None, node_mask_s=atom_mask_s, 
        #             edge_mask_s=edge_mask_s, n_nodes_s=n_nodes_s, node_mask_t=atom_mask_t, edge_mask_t=edge_mask_t, 
        #             n_nodes_t=n_nodes_t, x_s=atom_positions_s, x_t=atom_positions_t)
        pred = model(h0=[one_hot_s, one_hot_t], x=[atom_positions_s, atom_positions_t], all_edges=[edges_s, edges_t],
                    all_edge_attr=[None, None], node_masks=[atom_mask_s, atom_mask_t],
                    edge_masks=[edge_mask_s, edge_mask_t], n_nodes=[n_nodes_s, n_nodes_t])
        loss = loss_func(pred, label)
        error += loss.detach()
    return error

training_loss = []
testing_loss = []

for epoch in range(1, n_epochs):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    training_loss.append(train_acc.detach().numpy())
    testing_loss.append(test_acc.detach().numpy())
    print(f'Epoch: {epoch:03d}, Train MSE: {train_acc:.4f}, Test MSE: {test_acc:.4f}')

plt.figure(figsize=(12,6))
plt.plot(range(1, n_epochs), training_loss,label='training')
plt.plot(range(1, n_epochs), testing_loss,label='testing')
plt.ylabel('MSE')
plt.xlabel('Epochs')
plt.legend()

Processing...
[15:54:48] UFFTYPER: Unrecognized atom type: *_ (3)
[15:54:48] UFFTYPER: Unrecognized atom type: *_ (5)
[15:54:48] Molecule does not have explicit Hs. Consider calling AddHs()
[15:54:48] UFFTYPER: Unrecognized atom type: *_ (0)
[15:54:48] UFFTYPER: Unrecognized atom type: *_ (3)
[15:54:48] Molecule does not have explicit Hs. Consider calling AddHs()
[15:54:48] UFFTYPER: Unrecognized atom type: *_ (0)
[15:54:48] UFFTYPER: Unrecognized atom type: *_ (3)
[15:54:48] Molecule does not have explicit Hs. Consider calling AddHs()
[15:54:48] UFFTYPER: Unrecognized atom type: *_ (0)
[15:54:48] UFFTYPER: Unrecognized atom type: *_ (6)
[15:54:48] Molecule does not have explicit Hs. Consider calling AddHs()
[15:54:48] UFFTYPER: Unrecognized atom type: *_ (0)
[15:54:48] UFFTYPER: Unrecognized atom type: *_ (3)
[15:54:48] Molecule does not have explicit Hs. Consider calling AddHs()
[15:54:48] UFFTYPER: Unrecognized atom type: *_ (0)
[15:54:48] UFFTYPER: Unrecognized atom type: *_ (3)
[1

In [9]:
for data in valid_loader:
    dense_positions_s, atom_mask_s = tg.utils.to_dense_batch(data.positions_s, data.positions_s_batch)
    dense_positions_t, atom_mask_t = tg.utils.to_dense_batch(data.positions_t, data.positions_t_batch)
    batch_size_s, n_nodes_s, _ = dense_positions_s.size()
    batch_size_t, n_nodes_t, _ = dense_positions_t.size()
    atom_positions_s = dense_positions_s.view(batch_size_s * n_nodes_s, -1).to(device, dtype)
    atom_positions_t = dense_positions_t.view(batch_size_t * n_nodes_t, -1).to(device, dtype)
    edge_mask_s = atom_mask_s.unsqueeze(1) * atom_mask_s.unsqueeze(2)
    diag_mask = ~torch.eye(edge_mask_s.size(1), dtype=torch.bool).unsqueeze(0)
    edge_mask_s *= diag_mask
    edge_mask_s = edge_mask_s.view(batch_size_s * n_nodes_s * n_nodes_s, 1).to(device)
    edge_mask_t = atom_mask_t.unsqueeze(1) * atom_mask_t.unsqueeze(2)
    diag_mask = ~torch.eye(edge_mask_t.size(1), dtype=torch.bool).unsqueeze(0)
    edge_mask_t *= diag_mask
    edge_mask_t = edge_mask_t.view(batch_size_t * n_nodes_t * n_nodes_t, 1).to(device)
    atom_mask_s = atom_mask_s.view(batch_size_s * n_nodes_s, -1).to(device)
    atom_mask_t = atom_mask_t.view(batch_size_t * n_nodes_t, -1).to(device)
    one_hot_s, one_hot_s_mask = tg.utils.to_dense_batch(data.x_s, data.x_s_batch)
    one_hot_t, one_hot_s_mask = tg.utils.to_dense_batch(data.x_t, data.x_t_batch)
    one_hot_s = one_hot_s.view(batch_size_s * n_nodes_s, -1).to(device)
    one_hot_t = one_hot_t.view(batch_size_t * n_nodes_t, -1).to(device)
    edges_s = get_adj_matrix(n_nodes_s, batch_size_s, device)
    edges_t = get_adj_matrix(n_nodes_t, batch_size_t, device)
    label = data.y.to(device, dtype)
    
    pred = model(h0=[one_hot_s, one_hot_t], x=[atom_positions_s, atom_positions_t], all_edges=[edges_s, edges_t],
                all_edge_attr=[None, None], node_masks=[atom_mask_s, atom_mask_t],
                edge_masks=[edge_mask_s, edge_mask_t], n_nodes=[n_nodes_s, n_nodes_t])
    break

In [4]:
# clear the processed dataset
shutil.rmtree('./processed')

n_epochs  = 2
device = torch.device("cpu")
dtype = torch.float32
dat = Cloud_Point_Dataset(root='.')

batch_size = 16

dat.shuffle()
train_dataset = dat[:4000]
test_dataset = dat[4000:5000]
valid_dataset = dat[5000:]
train_loader = DataLoader(train_dataset, batch_size=batch_size, follow_batch=['x_s', 'x_t', 'positions_s', 'positions_t'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, follow_batch=['x_s', 'x_t','positions_s', 'positions_t'], shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, follow_batch=['x_s', 'x_t','positions_s', 'positions_t'], shuffle=False)

# model = MEGNN(n_graphs=2, in_node_nf=len(dat.elements), in_edge_nf=0, hidden_nf=128, device=device, n_layers=7, coords_weight=1.0,
#              attention=False, node_attr=1)
model = PairEGNN(in_node_nf=len(dat.elements), in_edge_nf=0, hidden_nf=128, device=device, n_layers=7, coords_weight=1.0,
             attention=False, node_attr=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()

def train():
    model.train()
    epoch_loss = 0
    for data in train_loader:  # Iterate in batches over the training dataset.
        dense_positions_s, atom_mask_s = tg.utils.to_dense_batch(data.positions_s, data.positions_s_batch)
        dense_positions_t, atom_mask_t = tg.utils.to_dense_batch(data.positions_t, data.positions_t_batch)
        batch_size_s, n_nodes_s, _ = dense_positions_s.size()
        batch_size_t, n_nodes_t, _ = dense_positions_t.size()
        atom_positions_s = dense_positions_s.view(batch_size_s * n_nodes_s, -1).to(device, dtype)
        atom_positions_t = dense_positions_t.view(batch_size_t * n_nodes_t, -1).to(device, dtype)
        edge_mask_s = atom_mask_s.unsqueeze(1) * atom_mask_s.unsqueeze(2)
        diag_mask = ~torch.eye(edge_mask_s.size(1), dtype=torch.bool).unsqueeze(0)
        edge_mask_s *= diag_mask
        edge_mask_s = edge_mask_s.view(batch_size_s * n_nodes_s * n_nodes_s, 1).to(device)
        edge_mask_t = atom_mask_t.unsqueeze(1) * atom_mask_t.unsqueeze(2)
        diag_mask = ~torch.eye(edge_mask_t.size(1), dtype=torch.bool).unsqueeze(0)
        edge_mask_t *= diag_mask
        edge_mask_t = edge_mask_t.view(batch_size_t * n_nodes_t * n_nodes_t, 1).to(device)
        atom_mask_s = atom_mask_s.view(batch_size_s * n_nodes_s, -1).to(device)
        atom_mask_t = atom_mask_t.view(batch_size_t * n_nodes_t, -1).to(device)
        one_hot_s, one_hot_s_mask = tg.utils.to_dense_batch(data.x_s, data.x_s_batch)
        one_hot_t, one_hot_s_mask = tg.utils.to_dense_batch(data.x_t, data.x_t_batch)
        one_hot_s = one_hot_s.view(batch_size_s * n_nodes_s, -1).to(device)
        one_hot_t = one_hot_t.view(batch_size_t * n_nodes_t, -1).to(device)
        edges_s = get_adj_matrix(n_nodes_s, batch_size_s, device)
        edges_t = get_adj_matrix(n_nodes_t, batch_size_t, device)
        label = data.y.to(device, dtype)
        
        pred = model(h0_s=one_hot_s, h0_t=one_hot_t, edges_s=edges_s, edges_t=edges_t, edge_attr=None, node_mask_s=atom_mask_s, 
                    edge_mask_s=edge_mask_s, n_nodes_s=n_nodes_s, node_mask_t=atom_mask_t, edge_mask_t=edge_mask_t, 
                    n_nodes_t=n_nodes_t, x_s=atom_positions_s, x_t=atom_positions_t)
        loss = loss_func(pred, label)
        epoch_loss += loss
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    return epoch_loss

def test(loader):
    model.eval()

    squared_error = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        dense_positions_s, atom_mask_s = tg.utils.to_dense_batch(data.positions_s, data.positions_s_batch)
        dense_positions_t, atom_mask_t = tg.utils.to_dense_batch(data.positions_t, data.positions_t_batch)
        batch_size_s, n_nodes_s, _ = dense_positions_s.size()
        batch_size_t, n_nodes_t, _ = dense_positions_t.size()
        atom_positions_s = dense_positions_s.view(batch_size_s * n_nodes_s, -1).to(device, dtype)
        atom_positions_t = dense_positions_t.view(batch_size_t * n_nodes_t, -1).to(device, dtype)
        edge_mask_s = atom_mask_s.unsqueeze(1) * atom_mask_s.unsqueeze(2)
        diag_mask = ~torch.eye(edge_mask_s.size(1), dtype=torch.bool).unsqueeze(0)
        edge_mask_s *= diag_mask
        edge_mask_s = edge_mask_s.view(batch_size_s * n_nodes_s * n_nodes_s, 1).to(device)
        edge_mask_t = atom_mask_t.unsqueeze(1) * atom_mask_t.unsqueeze(2)
        diag_mask = ~torch.eye(edge_mask_t.size(1), dtype=torch.bool).unsqueeze(0)
        edge_mask_t *= diag_mask
        edge_mask_t = edge_mask_t.view(batch_size_t * n_nodes_t * n_nodes_t, 1).to(device)
        atom_mask_s = atom_mask_s.view(batch_size_s * n_nodes_s, -1).to(device)
        atom_mask_t = atom_mask_t.view(batch_size_t * n_nodes_t, -1).to(device)
        one_hot_s, one_hot_s_mask = tg.utils.to_dense_batch(data.x_s, data.x_s_batch)
        one_hot_t, one_hot_s_mask = tg.utils.to_dense_batch(data.x_t, data.x_t_batch)
        one_hot_s = one_hot_s.view(batch_size_s * n_nodes_s, -1).to(device)
        one_hot_t = one_hot_t.view(batch_size_t * n_nodes_t, -1).to(device)
        edges_s = get_adj_matrix(n_nodes_s, batch_size_s, device)
        edges_t = get_adj_matrix(n_nodes_t, batch_size_t, device)
        label = data.y.to(device, dtype)
        
        pred = model(h0_s=one_hot_s, h0_t=one_hot_t, edges_s=edges_s, edges_t=edges_t, edge_attr=None, node_mask_s=atom_mask_s, 
                    edge_mask_s=edge_mask_s, n_nodes_s=n_nodes_s, node_mask_t=atom_mask_t, edge_mask_t=edge_mask_t, 
                    n_nodes_t=n_nodes_t, x_s=atom_positions_s, x_t=atom_positions_t)
        res = data.y - pred
        
        squared_error.append(torch.pow(res, 2).sum())

    return sum(squared_error) / len(squared_error)

training_loss = []
testing_loss = []

for epoch in range(1, n_epochs):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    training_loss.append(train_acc.detach().numpy())
    testing_loss.append(test_acc.detach().numpy())
    print(f'Epoch: {epoch:03d}, Train MSE: {train_acc:.4f}, Test MSE: {test_acc:.4f}')

Processing...
[13:56:55] UFFTYPER: Unrecognized atom type: *_ (3)
[13:56:55] UFFTYPER: Unrecognized atom type: *_ (5)
[13:56:55] Molecule does not have explicit Hs. Consider calling AddHs()
[13:56:55] UFFTYPER: Unrecognized atom type: *_ (0)
[13:56:55] UFFTYPER: Unrecognized atom type: *_ (3)
[13:56:55] Molecule does not have explicit Hs. Consider calling AddHs()
[13:56:55] UFFTYPER: Unrecognized atom type: *_ (0)
[13:56:55] UFFTYPER: Unrecognized atom type: *_ (3)
[13:56:55] Molecule does not have explicit Hs. Consider calling AddHs()
[13:56:55] UFFTYPER: Unrecognized atom type: *_ (0)
[13:56:55] UFFTYPER: Unrecognized atom type: *_ (6)
[13:56:55] Molecule does not have explicit Hs. Consider calling AddHs()
[13:56:55] UFFTYPER: Unrecognized atom type: *_ (0)
[13:56:55] UFFTYPER: Unrecognized atom type: *_ (3)
[13:56:55] Molecule does not have explicit Hs. Consider calling AddHs()
[13:56:55] UFFTYPER: Unrecognized atom type: *_ (0)
[13:56:55] UFFTYPER: Unrecognized atom type: *_ (3)
[1